In [1]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import joblib
import string
import numpy as np
import os

In [3]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
from sklearn.metrics import f1_score,precision_score,confusion_matrix,recall_score,accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier,GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import classification_report

In [4]:
import spacy
spacy.load("en_core_web_sm")

In [5]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Lucas\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Lucas\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Lucas\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [6]:
data = pd.read_csv(r"C:\Users\Lucas\OneDrive\Desktop\Resume Project\UpdatedResumeDataSet.csv")
data.head()

,Category,Resume
0,Data Science,Skills * Programming Languages: Python (pandas...
1,Data Science,Education Details \r\nMay 2013 to May 2017 B.E...
2,Data Science,"Areas of Interest Deep Learning, Control Syste..."
3,Data Science,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...
4,Data Science,"Education Details \r\n MCA YMCAUST, Faridab..."


In [7]:
# Remove punctuation
import string 
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [8]:
# function to remove punctuation
def remove_punct(text):
    return "".join([char for char in text if char not in string.punctuation])


In [9]:
# tokenize: converted sentence into words
def tokenize(text):
    return re.split('\W+', text.lower())

<>:3: SyntaxWarning: invalid escape sequence '\W'
<>:3: SyntaxWarning: invalid escape sequence '\W'
C:\Users\Lucas\AppData\Local\Temp\ipykernel_11948\3011256663.py:3: SyntaxWarning: invalid escape sequence '\W'
  return re.split('\W+', text.lower())


In [10]:
# removing stopwords
stopword = nltk.corpus.stopwords.words('english')

In [11]:
stopword = set(stopwords.words('english'))

In [12]:
def remove_stopwords(tokens):
    return [word for word in tokens if word not in stopword]

In [13]:
# steming: It helps reduce a word to its stem form. For example: “Entitling” or “Entitled” -----> become “Entitl.”

ps = nltk.PorterStemmer()
def stemming(tokenized_text):
    text = [ps.stem(word) for word in tokenized_text]
    return text 

In [14]:
# lemmatizing: This practice is more robust than stemming because it uses a dictionary-based approach to the root word.
# For example, “Entitling” or “Entitled” become “Entitle.”
wn = nltk.WordNetLemmatizer()

def lemmatize(tokens):
    return [wn.lemmatize(word) for word in tokens]

In [15]:
data['Resume_text_clean'] = data['Resume'].apply(remove_punct)
data['Resume_text_tokenized'] = data['Resume_text_clean'].apply(tokenize)
data['Resume_text_nostop'] = data['Resume_text_tokenized'].apply(remove_stopwords)
data['Resume_text_lemmatized'] = data['Resume_text_nostop'].apply(lemmatize)

In [16]:
vectorizer = TfidfVectorizer(stop_words='english')

In [18]:
data['processed_resume'] = data['Resume_text_lemmatized'].apply(lambda x: ' '.join(x))

In [19]:
data.head()

,Category,Resume,Resume_text_clean,Resume_text_tokenized,Resume_text_nostop,Resume_text_lemmatized,processed_resume
0,Data Science,Skills * Programming Languages: Python (pandas...,Skills Programming Languages Python pandas nu...,"[skills, programming, languages, python, panda...","[skills, programming, languages, python, panda...","[skill, programming, language, python, panda, ...",skill programming language python panda numpy ...
1,Data Science,Education Details \r\nMay 2013 to May 2017 B.E...,Education Details \r\nMay 2013 to May 2017 BE ...,"[education, details, may, 2013, to, may, 2017,...","[education, details, may, 2013, may, 2017, uit...","[education, detail, may, 2013, may, 2017, uitr...",education detail may 2013 may 2017 uitrgpv dat...
2,Data Science,"Areas of Interest Deep Learning, Control Syste...",Areas of Interest Deep Learning Control System...,"[areas, of, interest, deep, learning, control,...","[areas, interest, deep, learning, control, sys...","[area, interest, deep, learning, control, syst...",area interest deep learning control system des...
3,Data Science,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...,Skills â¢ R â¢ Python â¢ SAP HANA â¢ Table...,"[skills, â, r, â, python, â, sap, hana, â, tab...","[skills, â, r, â, python, â, sap, hana, â, tab...","[skill, â, r, â, python, â, sap, hana, â, tabl...",skill â r â python â sap hana â tableau â sap ...
4,Data Science,"Education Details \r\n MCA YMCAUST, Faridab...",Education Details \r\n MCA YMCAUST Faridaba...,"[education, details, mca, ymcaust, faridabad, ...","[education, details, mca, ymcaust, faridabad, ...","[education, detail, mca, ymcaust, faridabad, h...",education detail mca ymcaust faridabad haryana...


In [20]:
X = vectorizer.fit_transform(data['processed_resume'])
y = data['Category']

In [21]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=45, stratify=y)

In [22]:
#model = SVC(kernel='linear', probability=True) model.fit(X_train, y_train)
lgbm = LGBMClassifier()
lgbm.fit(X_train,y_train)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.029302 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 16268
[LightGBM] [Info] Number of data points in the train set: 769, number of used features: 1311
[LightGBM] [Info] Start training from score -3.872502
[LightGBM] [Info] Start training from score -3.277795
[LightGBM] [Info] Start training from score -3.600569
[LightGBM] [Info] Start training from score -3.179355
[LightGBM] [Info] Start training from score -3.509597
[LightGBM] [Info] Start training from score -3.700652
[LightGBM] [Info] Start training from score -3.179355
[LightGBM] [Info] Start training from score -3.386994
[LightGBM] [Info] Start training from score -2.860901
[LightGBM] [Info] Start training from score -3.554049
[LightGBM] [Info] Start training from score -3.179355
[LightGBM] [Info] Start training from score -3.467037
[LightGBM] [Info] Start training from score -3.089743
[LightGB

LGBMClassifier()

In [23]:
# y_pred = model.predict(X_test)  accuracy = accuracy_score(y_test, y_pred)  print(f"Accuracy: {accuracy:.4f}")
predict9 = lgbm.predict(X_test)
accuracy_9 = accuracy_score(y_test,predict9)
print(accuracy_9)

1.0


In [24]:
#print(confusion_matrix(y_test, y_pred))
cm9 = confusion_matrix(y_test,predict9)


In [25]:
#print(classification_report(y_test, y_pred))
print(classification_report(y_test,predict9))

                           precision    recall  f1-score   support

                 Advocate       1.00      1.00      1.00         4
                     Arts       1.00      1.00      1.00         7
       Automation Testing       1.00      1.00      1.00         5
               Blockchain       1.00      1.00      1.00         8
         Business Analyst       1.00      1.00      1.00         5
           Civil Engineer       1.00      1.00      1.00         5
             Data Science       1.00      1.00      1.00         8
                 Database       1.00      1.00      1.00         7
          DevOps Engineer       1.00      1.00      1.00        11
         DotNet Developer       1.00      1.00      1.00         6
            ETL Developer       1.00      1.00      1.00         8
   Electrical Engineering       1.00      1.00      1.00         6
                       HR       1.00      1.00      1.00         9
                   Hadoop       1.00      1.00      1.00     

In [26]:
joblib.dump(vectorizer, 'vectorizer.pkl')
joblib.dump(lgbm, 'lgbm_model.pkl')

['lgbm_model.pkl']

In [27]:
vectorizer = joblib.load('vectorizer.pkl')
model = joblib.load('lgbm_model.pkl')

In [28]:
sample_resume = ["Your sample resume text here"]
sample_resume_transformed = vectorizer.transform(sample_resume)
sample_prediction = model.predict(sample_resume_transformed)
print(f"Predicted Category: {sample_prediction[0]}")

Predicted Category: ETL Developer
